In [1]:
import numpy as np
import igl
import meshplot as mp
mp.offline()

In [2]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [3]:
# groups vertices into a nxnxn grid
def create_grid(minx,miny,minz,maxx,maxy,maxz,points,n):
    # create the size of grid cell
    disx = (maxx-minx)/n
    disy = (maxy-miny)/n
    disz = (maxz-minz)/n
    #print(disx,disy,disz)
    #print(minx,miny,minz)
    #print(maxx,maxy,maxz)
    
    # create the grid
    grid = [[] for i in range(216)]
    
    # sort the list of points into buckets
    for i in range(len(points)):
        #print(points[i])
        # find which x bucket point falls into
        temp_x = minx + disx
        xpos = 0
        while(points[i][0] > temp_x and xpos < n-1):
            temp_x += disx
            xpos += 1
        # find which y bucket point falls into
        temp_y = miny + disy
        ypos = 0
        while(points[i][1] > temp_y and ypos < n-1):
            temp_y += disy
            ypos += 1
        # find which z bucket point falls into
        temp_z = minz + disz
        zpos = 0
        while(points[i][2] > temp_z and zpos < n-1):
            temp_z += disz
            zpos += 1
        #print("x",xpos,temp_x,points[i][0])
        #print("y",ypos,temp_y,points[i][1])
        #print("z",zpos,temp_z,points[i][2])
        #print(n)
        #print(xpos + ypos*n + zpos*n*n)
        sum = xpos + ypos*n + zpos*n*n
        #print(sum)
        grid[sum].append(i)
    #print(grid)
    #print(len(grid))
    return grid, (n-1 + (n-1)*n + (n-1)*n*n)+1 # number of cells

In [4]:
def find_closest_point(point,point_list):
    min_distance = 999
    index = -1
    for i in range(len(point_list)):
        xdist = point_list[i][0] - point[0]
        ydist = point_list[i][1] - point[1]
        zdist = point_list[i][2] - point[2]
        dist = np.sqrt(xdist*xdist + ydist*ydist + zdist*zdist)
        if(min_distance > dist):
            min_distance = dist
            index = i
    return index

In [5]:
def find_closest_point_improved(point,point_list,points_to_consider):
    min_distance = 999
    index = -1
    for i in range(len(points_to_consider)):
        xdist = point_list[points_to_consider[i]][0] - point[0]
        ydist = point_list[points_to_consider[i]][1] - point[1]
        zdist = point_list[points_to_consider[i]][2] - point[2]
        dist = np.sqrt(xdist*xdist + ydist*ydist + zdist*zdist)
        if(min_distance > dist):
            min_distance = dist
            index = points_to_consider[i]   
    return index

In [6]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
#print(pi.shape)
#print(ni.shape)
f_final = np.zeros((pi.shape[0]*3),dtype=pi.dtype) # input function
pi_final = np.zeros((pi.shape[0]*3,3),dtype=pi.dtype) # output of function
f = np.zeros((pi.shape[0]),dtype=pi.dtype) # input function
f_plus = np.zeros((pi.shape[0]),dtype=pi.dtype) # input function
f_minus = np.zeros((pi.shape[0]),dtype=pi.dtype) # input function
pi_plus = np.zeros((pi.shape[0],3),dtype=pi.dtype) # output of function
pi_minus = np.zeros((pi.shape[0],3),dtype=pi.dtype) # output of function

# calculate the min and max values for bounding box diagonal
minx = 100
miny = 100
minz = 100
maxx = -100
maxy = -100
maxz = -100
for i in range(len(pi)):
    if minx > pi[i][0]:
        minx = pi[i][0]
    if maxx < pi[i][0]:
        maxx = pi[i][0]
    if miny > pi[i][1]:
        miny = pi[i][1]
    if maxy < pi[i][1]:
        maxy = pi[i][1]
    if minz > pi[i][2]:
        minz = pi[i][2]
    if maxz < pi[i][2]:
        maxz = pi[i][2]
lenx = maxx-minx
leny = maxy-miny
lenz = maxz-minz
bblen = np.sqrt(lenx*lenx + leny*leny + lenz*lenz) # length of bounding box
eps = 0.01*bblen # default starting epsilon
#print(eps)
#print(pi_plus)
grid_size = 6
#create a grid mapping for the points
grid,grid_cells = create_grid(minx,miny,minz,maxx,maxy,maxz,pi,grid_size)
# loop though pi to create all the pi+ and pi-
for i in range(len(pi)):
    closest_point_index = -1
    loop_multiplier = 1
    
    # find which grid the point belongs to
    grid_index_found = -1
    for j in range(len(grid)):
        for k in range(len(grid[j])):
            # check for a match
            if i == grid[j][k]:
                #print(i,grid[j],j)
                grid_index_found = j
    # add all neightboring grids
    grid_list = []
    grid_list.append(grid_index_found)
    # add adjacent x,y,z grids
    x_size = 2
    y_size = 2
    z_size = 2
    for j in range(0,x_size):
        for k in range(0,y_size):
            for l in range(0,z_size):
                #print(grid_index_found + j + k*grid_size + l*grid_size*grid_size)
                #print(j,k,l)
                grid_list.append(grid_index_found + j + k*grid_size + l*grid_size*grid_size)
                grid_list.append(grid_index_found - j + k*grid_size + l*grid_size*grid_size)
                grid_list.append(grid_index_found + j - k*grid_size + l*grid_size*grid_size)
                grid_list.append(grid_index_found + j + k*grid_size - l*grid_size*grid_size)
                grid_list.append(grid_index_found - j - k*grid_size + l*grid_size*grid_size)
                grid_list.append(grid_index_found - j + k*grid_size - l*grid_size*grid_size)
                grid_list.append(grid_index_found + j - k*grid_size - l*grid_size*grid_size)
                grid_list.append(grid_index_found - j - k*grid_size - l*grid_size*grid_size)
                
    # remove duplicates
    grid_list = set(grid_list)
    grid_list = list(grid_list)
    #print("New list",grid_list,grid_index_found)
    
    # remove invalid points
    new_grid_list = []
    for j in range(len(grid_list)):
        if grid_list[j] >= 0 and grid_list[j] < grid_cells:
            new_grid_list.append(grid_list[j])
    #print("Newest list",new_grid_list)
    
    # create a temporary list which has only indices of points to consider
    new_point_list = []
    for j in range(len(new_grid_list)):
        for k in range(len(grid[new_grid_list[j]])):
            #print(new_grid_list[j])
            #print(grid[new_grid_list[j]][k])
            new_point_list.append(grid[new_grid_list[j]][k])
    #print(new_point_list)
    while(closest_point_index != i):
        # calculate new point
        pi_plus_temp = pi[i] + (eps/loop_multiplier) * ni[i]
        #print(pi_plus_temp,pi[i],loop_multiplier)
        
        # toggle between brute force and grid method
        #closest_point_index = find_closest_point(pi_plus_temp,pi)
        closest_point_index = find_closest_point_improved(pi_plus_temp,pi,new_point_list)
        
        if(closest_point_index != i):
            loop_multiplier*=2
    
    closest_point_index2 = -1
    loop_multiplier2 = 1
    while(closest_point_index2 != i):
        # calculate new point
        pi_minus_temp = pi[i] - (eps/loop_multiplier2) * ni[i]
        closest_point_index2 = find_closest_point(pi_minus_temp,pi)
        if(closest_point_index2 != i):
            loop_multiplier2*=2
    f_plus[i] = eps
    pi_plus[i] = pi_plus_temp
    f_minus[i] = -eps
    pi_minus[i] = pi_minus_temp
#print(pi_plus)
#print(pi_minus)

# concatanate the pi, f values
for i in range(len(pi)):
    pi_final[i*3] = pi[(i)]
    pi_final[i*3+1] = pi_plus[(i)]
    pi_final[i*3+2] = pi_minus[(i)]
    f_final[i*3] = f[(i)]
    f_final[i*3+1] = f_plus[(i)]
    f_final[i*3+2] = f_minus[(i)]
ind = np.zeros((f_final.shape[0],3))
#print(f_final.shape)
ind[f_final > 0] = [255,0,0]
ind[f_final == 0] = [0,0,255]
ind[f_final < 0] = [0,255,0]
#print(ind)
mp.plot(pi_final, c=ind, shading={"point_size": 8},filename = "Cat")

Plot saved to file Cat.html.


# MLS function

In [41]:
# given code consolidated, required to run to make other code work
# Parameters
bbox_min = np.array([-1., -1., -1.])
bbox_max = np.array([1., 1., 1.])
bbox_diag = np.linalg.norm(bbox_max - bbox_min)

n = 20
# Generate grid n x n x n

x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

#Compute implicit sphere function
center = np.array([0., 0., 0.])
radius = 1
fx = np.linalg.norm(x-center, axis=1) - radius
# Treshold fx to visualize inside outside

ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1
mp.plot(x, c=ind, shading={"point_size": 0.1,"width": 800, "height": 800},filename = "Example_sphere")

Plot saved to file Example_sphere.html.


In [31]:
# Parameters modified to handle non axis aligned
bbox_min = np.array([minx, miny, minz])
bbox_max = np.array([maxx, maxy, maxz])
bbox_diag = np.linalg.norm(bbox_max - bbox_min)

In [32]:
def closest_points(point, point_list, h):
    value_list = []
    for i in range(len(point_list)):
        # calculate the distance between point and point_list index value
        x_value = point_list[i][0] - point[0]
        y_value = point_list[i][1] - point[1]
        z_value = point_list[i][2] - point[2]
        #print(x_value, y_value, z_value)
        temp_distance = np.sqrt(x_value*x_value + y_value*y_value + z_value*z_value)
        if(temp_distance < h):
            value_list.append(i)
            #print(temp_distance)
    return value_list

In [33]:
def closest_points_improved(point, point_list, h,points_to_consider):
    value_list = []
    for i in range(len(points_to_consider)):
        # calculate the distance between point and point_list index value
        x_value = point_list[points_to_consider[i]][0] - point[0]
        y_value = point_list[points_to_consider[i]][1] - point[1]
        z_value = point_list[points_to_consider[i]][2] - point[2]
        #print(x_value, y_value, z_value)
        temp_distance = np.sqrt(x_value*x_value + y_value*y_value + z_value*z_value)
        if(temp_distance < h):
            value_list.append(points_to_consider[i])
            #print(temp_distance)
    return value_list

In [34]:
def wendland_weight(x,c,h):
    # calculate the length
    x_value = x[0]-c[0]
    y_value = x[1]-c[1]
    z_value = x[2]-c[2]
    length = np.sqrt(x_value*x_value + y_value*y_value + z_value*z_value)
    #print(x_value,y_value,z_value,length)
    #print(x[0],c[0],x[1],c[1],x[2],c[2])
    if(length < h):
        quad_value = 1-(length/h)
        weight = quad_value * quad_value * quad_value * quad_value * (4*(length/h)+1)
    else:
        weight = 0
    #print(weight)
    return weight

In [35]:
def wendlandRadius(point,point_list,point_list_full,inside_point_index,h):
    # construct matrix W
    W = np.zeros((len(inside_point_index),3),dtype = "double")
    
    # use the inside point indices to get the point_list_full points
    '''if (len(inside_point_index) >= 1):
        print("new_radius")'''
    #print("W",W.shape)
    for i in range(len(inside_point_index)):
        #print(inside_point_index[i])
        # convert the indexes from pi to hit all three associated values in pi_plus
        translated_index, translated_index_plus, translated_index_minus = inside_point_index[i] * 3,inside_point_index[i] * 3 + 1,inside_point_index[i] * 3 + 2 
        #print(translated_index, translated_index_plus, translated_index_minus)
        #print(point_list[inside_point_index[i]])
        #print(point_list_full[translated_index],point_list_full[translated_index_plus],point_list_full[translated_index_minus])
        #print(point)
        c1 = wendland_weight(point,point_list_full[translated_index],h)
        c2 = wendland_weight(point,point_list_full[translated_index_plus],h)
        c3 = wendland_weight(point,point_list_full[translated_index_minus],h)
        W[i][0] = c1
        W[i][1] = c2
        W[i][2] = c3
    return W


In [36]:
def polyDegree(point,point_list,value_list,h):
    # construct matrix B
    B = np.zeros((len(value_list),10),dtype = "double")
    
    for i in range(len(value_list)):
        x_value = point_list[value_list[i]][0]
        y_value = point_list[value_list[i]][1]
        z_value = point_list[value_list[i]][2]
        #print(x_value,y_value,z_value)
        B[i][0] = 1
        B[i][1] = x_value
        B[i][2] = y_value
        B[i][3] = z_value
        B[i][4] = x_value * x_value
        B[i][5] = x_value * y_value
        B[i][6] = x_value * z_value
        B[i][7] = y_value * y_value
        B[i][8] = y_value * z_value
        B[i][9] = z_value * z_value
    #print(B.shape)
    BT = np.transpose(B)
    #print(BT.shape)
    #print(BT)
    return BT

In [37]:
x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)
fxx = np.zeros_like(fx)

h = 25 # distance point must be to test point
# calculate how many grids it could touch in the worst case x,y,z plane
disx = (maxx-minx)/grid_size
disy = (maxy-miny)/grid_size
disz = (maxz-minz)/grid_size
x_size = 2
y_size = 2
z_size = 2
while disx < h:
    disx += (maxx-minx)/grid_size
    x_size+= 1
while disy < h:
    disy += (maxy-miny)/grid_size
    y_size+= 1
while disz < h:
    disz += (maxz-minz)/grid_size
    z_size+= 1
#print(disx,disy,disz)
#print(x_size,y_size,z_size)
# loop over all cells
for i in range(len(x)):

    # check all points that are close enough, add them to a list of values to calculate local basis
    # find which grid cell the point x belongs to
    temp_x = minx + disx
    xpos = 0
    while(x[i][0] > temp_x and xpos < n-1):
        temp_x += disx
        xpos += 1
     # find which y bucket point falls into
    temp_y = miny + disy
    ypos = 0
    while(x[i][1] > temp_y and ypos < n-1):
        temp_y += disy
        ypos += 1
    # find which z bucket point falls into
    temp_z = minz + disz
    zpos = 0
    while(x[i][2] > temp_z and zpos < n-1):
        temp_z += disz
        zpos += 1
    grid_index_found = xpos + ypos*grid_size + zpos*grid_size*grid_size
    # add all neightboring grids
    grid_list = []
    grid_list.append(grid_index_found)
    for j in range(0,x_size):
        for k in range(0,y_size):
            for l in range(0,z_size):
                #print(grid_index_found + j + k*grid_size + l*grid_size*grid_size)
                #print(j,k,l)
                grid_list.append(grid_index_found + j + k*grid_size + l*grid_size*grid_size)
                grid_list.append(grid_index_found - j + k*grid_size + l*grid_size*grid_size)
                grid_list.append(grid_index_found + j - k*grid_size + l*grid_size*grid_size)
                grid_list.append(grid_index_found + j + k*grid_size - l*grid_size*grid_size)
                grid_list.append(grid_index_found - j - k*grid_size + l*grid_size*grid_size)
                grid_list.append(grid_index_found - j + k*grid_size - l*grid_size*grid_size)
                grid_list.append(grid_index_found + j - k*grid_size - l*grid_size*grid_size)
                grid_list.append(grid_index_found - j - k*grid_size - l*grid_size*grid_size)
                
    # remove duplicates
    grid_list = set(grid_list)
    grid_list = list(grid_list)
    #print("New list",grid_list,grid_index_found)
    
    # remove invalid points
    new_grid_list = []
    for j in range(len(grid_list)):
        if grid_list[j] >= 0 and grid_list[j] < grid_cells:
            new_grid_list.append(grid_list[j])
    #print("Newest list",new_grid_list)
    
    # create a temporary list which has only indices of points to consider
    new_point_list = []
    for j in range(len(new_grid_list)):
        for k in range(len(grid[new_grid_list[j]])):
            #print(new_grid_list[j])
            #print(grid[new_grid_list[j]][k])
            new_point_list.append(grid[new_grid_list[j]][k])
            
    # toggle between brute force and grid        
    #value_list = closest_points(x[i],pi,h)
    value_list = closest_points_improved(x[i],pi,h,new_point_list)
    
    #BT = polyDegree(x[i],pi,value_list,h)
    #W = wendlandRadius(x[i],pi,pi_final,value_list,h)
    #print("BT",BT.shape)
    #print(BT)
    #print("W",W.shape)
    #print(W)
    #BT_W = np.matmul(BT,W)
    #print(BT_W.shape)
    # construct d
    #print(f_final.shape)
    # for every point, calculate its wendland weight and multiply it by the value, then sum the value to evaluate at the point
    total_weight = 0
    # set starting weight
    if len(value_list) >= 1:
        total_weight = 0
        for j in range(len(value_list)):
            #print(value_list[i])
            translated_index, translated_index_plus, translated_index_minus = value_list[j] * 3,value_list[j] * 3 + 1,value_list[j] * 3 + 2 
            #print(x[i],pi_final[translated_index])
            c1 = wendland_weight(x[i],pi_final[translated_index],h)
            c2 = wendland_weight(x[i],pi_final[translated_index_plus],h)
            c3 = wendland_weight(x[i],pi_final[translated_index_minus],h)
            #print(c1,c2,c3)
            #print(value_list)
            c1 *= f_final[0]
            c2 *= f_final[1]
            c3 *= f_final[2]
            total_weight = total_weight + c1 + c2 + c3
    else:
        total_weight = 999
    #print(total_weight)
    fxx[i] = total_weight
# 1 x y z x2 xy xz y2 yz z2
print("finished")

finished


In [38]:
# Treshold fx to visualize inside outside
ind = np.zeros_like(fxx)
ind[fxx >= 0] = 1
ind[fxx < 0] = -1
mp.plot(x, c=ind, shading={"point_size": 4,"width": 800, "height": 800},filename = "MLS")

Plot saved to file MLS.html.


In [39]:
# axis align the luigi
pi2, v2 = igl.read_triangle_mesh("data/luigi.off")
ni2 = igl.per_vertex_normals(pi2, v2)

x_sum = 0
y_sum = 0
z_sum = 0
# calculate the centroid
for i in range(len(pi2)):
    x_sum += pi2[i][0]
    y_sum += pi2[i][1]
    z_sum += pi2[i][2]
#print(x_sum,y_sum,z_sum,len(pi2))
x_sum /= len(pi2)
y_sum /= len(pi2)
z_sum /= len(pi2)
#print(x_sum,y_sum,z_sum,len(pi2))
centroid = np.zeros((1,3))
centroid[0][0] = x_sum
centroid[0][1] = y_sum
centroid[0][2] = z_sum

# calculate the vectors yi from the centroids
YT = np.zeros_like(pi2) # calculate the transpose, easier to think about
for i in range(len(YT)):
    YT[i][0] = pi2[i][0] - centroid[0][0]
    YT[i][1] = pi2[i][1] - centroid[0][1]
    YT[i][2] = pi2[i][2] - centroid[0][2] 
# construct the matrix Y by transposing the transpose
Y = YT.transpose()
#print(Y)
#print(YT)
S = np.dot(Y,YT)
print(S)

# calculate the length of the eigen vectors to get the smallest one
e1 = np.sqrt(S[0][0] * S[0][0] + S[1][0] * S[1][0] + S[2][0] * S[2][0])
e2 = np.sqrt(S[0][1] * S[0][1] + S[1][1] * S[1][1] + S[2][1] * S[2][1])
e3 = np.sqrt(S[0][2] * S[0][2] + S[1][2] * S[1][2] + S[2][2] * S[2][2])
e = 0
e_vector = np.zeros((3,1))
if(e1 < e2):
    e = e1
    e_vector[0] = S[0][0]
    e_vector[1] = S[1][0]
    e_vector[2] = S[2][0]
else:
    e = e2
    e_vector[0] = S[0][1]
    e_vector[1] = S[1][1]
    e_vector[2] = S[2][1]
if(e > e3):
    e = e3
    e_vector[0] = S[0][2]
    e_vector[1] = S[1][2]
    e_vector[2] = S[2][2]
print(e,e_vector)
e_norm = e_vector / np.linalg.norm(e_vector)
print(e_norm)
mp.plot(pi2, shading={"point_size": 8},filename = "Luigi")

[[1436556.27509452 1166595.41158901  765795.74822034]
 [1166595.41158901 1598937.37324176 1128053.6937445 ]
 [ 765795.74822034 1128053.6937445  1391163.24675363]]
1947892.0511879148 [[ 765795.74822034]
 [1128053.6937445 ]
 [1391163.24675363]]
[[0.39314075]
 [0.5791151 ]
 [0.71418909]]
Plot saved to file Luigi.html.


# Marching to extract surface

In [40]:
# Marcing tet to extract surface

sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
mp.plot(sv, sf, shading={"wireframe": True}, filename = "marching_example")

sv, sf, _, _ = igl.marching_tets(x, T, fxx, 0)
mp.plot(sv, sf, shading={"wireframe": True}, filename = "marching_output")

# remove all other faces except largest componenet
c = igl.face_components(sf)
largest_component = 0
largest_component_id = -1
for i in range (0, max(c + 1)):
    count = 0
    for j in range (0, len(c)):
        if(i == c[j]):
            count += 1
    print("Component number:", i, "Size of component:", count)
    if(largest_component < count):
        largest_component = count
        largest_component_id = i
print(largest_component_id)

# remove all faces that aren't part of the largest component
sf_new = np.zeros((largest_component,3))
ind = np.zeros((sf_new.shape[0],3),dtype = "int64")
add_index = 0
for i in range(len(c)):
    if c[i] == largest_component_id:
        sf_new[add_index][0] = sf[i][0]
        sf_new[add_index][1] = sf[i][1]
        sf_new[add_index][2] = sf[i][2]
        add_index += 1
for i in range(len(ind)):
    ind[i] = [255,0,0]
p = mp.plot(sv, sf_new,shading={"wireframe": True}, filename = "One_component")

Plot saved to file marching_example.html.
Plot saved to file marching_output.html.
Component number: 0 Size of component: 33114
0
Plot saved to file One_component.html.
